In [10]:
import cv2
import requests
from PIL import Image
import torch
import numpy as np


from matplotlib import pyplot as plt

from transformers import TrOCRProcessor, VisionEncoderDecoderModel

In [15]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-printed")
ocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-printed")

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.43G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
VIDEO_FILEPATH = 'testdata/20210111_DML2387.mp4'

In [17]:
video = cv2.VideoCapture(VIDEO_FILEPATH)
if not video.isOpened():
    raise RuntimeError('Video opening failed')

In [18]:
interesting_images = []
count = 0
step_size = 10
while video.isOpened():
    video.set(cv2.CAP_PROP_POS_FRAMES, count)
    count += step_size
    ret, frame = video.read()
    if not ret: break

    h, w, c = frame.shape
    frame = cv2.resize(frame, (w//4, h//4))
    frame = Image.fromarray(frame)
    pixel_values = processor(frame, return_tensors="pt").pixel_values
    generated_ids = ocr_model.generate(pixel_values)

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)
    if count > 500: break


video.release()
cv2.destroyAllWindows()

AMOUNT
TOTAL AMOUNT
TOTAL ALL ALL ALL ALL ALL ALL ALL ALL ALL ALL WITH ALL ALL ALL ALL ALL ALL
TOTAL AMOUNT
TOTAL AMOUNT
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL AMOUNT
TOTAL
TOTAL
TOTAL
BARRY
TOTAL AMOUNT
AMOUNT
TOTAL AMOUNT
TOTAL :
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL :
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL
TOTAL


In [ ]:
s = sorted(interesting_images, key=lambda t: t[1])

In [ ]:
for t in reversed(s):
    plt.imshow(t[0])
    plt.show()